# Exploratory Data Analysis for Car Price Prediction

This notebook performs exploratory data analysis on car sales advertisements data to prepare for building a prediction model for car prices.

## 1. Setup and Data Loading

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import missingno as msno
import warnings

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

In [ ]:
# Load the dataset
df = pd.read_csv('data/sales_ads_train.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
df.head()

## 2. Data Overview and Structure

In [ ]:
# Check column data types
df.info()

In [ ]:
# Statistical summary of numerical features
df.describe().T

In [ ]:
# Distribution of categorical features
cat_columns = df.select_dtypes(include=['object']).columns

for col in cat_columns[:5]:  # Display first 5 categorical columns
    print(f"\n{col} - {df[col].nunique()} unique values")
    print(df[col].value_counts().head())

## 3. Target Variable Analysis (Price)

In [ ]:
# Basic statistics of price
print(df['Cena'].describe())

# Visualize price distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['Cena'], kde=True)
plt.title('Price Distribution')
plt.xlabel('Price')

plt.subplot(1, 2, 2)
sns.histplot(np.log1p(df['Cena']), kde=True)
plt.title('Log-transformed Price Distribution')
plt.xlabel('Log(Price+1)')

plt.tight_layout()
plt.show()

In [ ]:
# Price distribution by currency
plt.figure(figsize=(10, 6))
sns.boxplot(x='Waluta', y='Cena', data=df)
plt.title('Price Distribution by Currency')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Missing Values Analysis

In [ ]:
# Check for missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing_values / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percent})
missing_df = missing_df[missing_df['Missing Values'] > 0]
missing_df

In [ ]:
# Visualize missing values patterns
plt.figure(figsize=(12, 8))
msno.matrix(df)
plt.title('Missing Values Matrix')
plt.show()

## 5. Numerical Features Analysis

In [ ]:
# Select numerical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_cols = [col for col in num_cols if col != 'ID']

# Correlation matrix
plt.figure(figsize=(14, 10))
correlation = df[num_cols].corr()
mask = np.triu(correlation)
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', mask=mask, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

In [ ]:
# Relationship between key numerical features and price
key_features = ['Rok_produkcji', 'Przebieg_km', 'Moc_KM', 'Pojemnosc_cm3']

plt.figure(figsize=(16, 12))
for i, feature in enumerate(key_features):
    plt.subplot(2, 2, i+1)
    plt.scatter(df[feature], df['Cena'], alpha=0.3)
    plt.title(f'{feature} vs Price')
    plt.xlabel(feature)
    plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

## 6. Categorical Features Analysis

In [ ]:
# Top car brands by average price
brand_price = df.groupby('Marka_pojazdu')['Cena'].agg(['mean', 'count'])
brand_price = brand_price[brand_price['count'] > 100].sort_values('mean', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(x=brand_price.index[:15], y='mean', data=brand_price[:15])
plt.title('Top 15 Car Brands by Average Price')
plt.xlabel('Brand')
plt.ylabel('Average Price')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Price distribution by car body type
plt.figure(figsize=(14, 8))
sns.boxplot(x='Typ_nadwozia', y='Cena', data=df)
plt.title('Price Distribution by Car Body Type')
plt.xlabel('Body Type')
plt.ylabel('Price')
plt.yscale('log')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Price distribution by fuel type
plt.figure(figsize=(14, 8))
sns.boxplot(x='Rodzaj_paliwa', y='Cena', data=df)
plt.title('Price Distribution by Fuel Type')
plt.xlabel('Fuel Type')
plt.ylabel('Price')
plt.yscale('log')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Feature Engineering Exploration

In [ ]:
# Create age feature
current_year = pd.to_datetime('today').year
df['car_age'] = current_year - df['Rok_produkcji']

# Explore price vs car age relationship
plt.figure(figsize=(12, 6))
sns.scatterplot(x='car_age', y='Cena', data=df, alpha=0.3)
plt.title('Price vs Car Age')
plt.xlabel('Car Age (years)')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Extract equipment features
# First check if Wyposazenie is in string format and contains lists
if df['Wyposazenie'].dtype == 'object' and df['Wyposazenie'].str.contains('\[').any():
    # Convert string representation of lists to actual lists
    try:
        df['equipment_list'] = df['Wyposazenie'].apply(lambda x: eval(x) if isinstance(x, str) else [])
        
        # Count equipment features
        df['equipment_count'] = df['equipment_list'].apply(len)
        
        # Plot distribution of equipment count
        plt.figure(figsize=(12, 6))
        sns.histplot(df['equipment_count'], bins=30, kde=True)
        plt.title('Distribution of Equipment Features Count')
        plt.xlabel('Number of Equipment Features')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Relationship between equipment count and price
        plt.figure(figsize=(12, 6))
        sns.scatterplot(x='equipment_count', y='Cena', data=df, alpha=0.3)
        plt.title('Price vs Equipment Features Count')
        plt.xlabel('Number of Equipment Features')
        plt.ylabel('Price')
        plt.grid(True, alpha=0.3)
        plt.show()
    except:
        print("Could not parse equipment list.")
else:
    print("Equipment column not in expected format.")

## 8. Outlier Detection and Analysis

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

# Detect price outliers
price_outliers = detect_outliers(df, 'Cena')
print(f"Number of price outliers: {len(price_outliers)}")
print(f"Percentage of price outliers: {len(price_outliers) / len(df) * 100:.2f}%")

# Display some statistics about outliers
if len(price_outliers) > 0:
    print("\nPrice outliers statistics:")
    print(price_outliers['Cena'].describe())

## 9. Data Preparation for Modeling

In [ ]:
# Create a copy of the dataframe for preprocessing
df_prep = df.copy()

# List of potentially important features for price prediction
features = [
    'Marka_pojazdu', 'Model_pojazdu', 'Rok_produkcji', 'Przebieg_km', 
    'Moc_KM', 'Pojemnosc_cm3', 'Rodzaj_paliwa', 'Naped',
    'Skrzynia_biegow', 'Typ_nadwozia', 'Stan', 'car_age'
]

# Check which features from our list are available
available_features = [f for f in features if f in df_prep.columns]
print(f"Available features for modeling: {available_features}")

In [ ]:
# Convert currency if needed
if 'Waluta' in df_prep.columns:
    # Check unique currencies
    print(f"Unique currencies: {df_prep['Waluta'].unique()}")
    
    # Note: For a real model, we would need to normalize all prices to a single currency
    # using appropriate exchange rates. For this EDA, we'll just note the currencies.

## 10. Key Findings and Insights

**Key findings from the EDA:**

1. **Price Distribution**: The target variable distribution and whether log transformation would be beneficial
2. **Missing Values**: Patterns and strategies for handling missing values
3. **Important Correlations**: Numerical features most correlated with price
4. **Categorical Impact**: Which categorical features have the strongest relationship with price
5. **Feature Engineering**: Effectiveness of engineered features like car age and equipment count
6. **Outliers**: Presence and potential impact of outliers on the model

## 11. Next Steps for Modeling

**Recommended next steps for building a price prediction model:**

1. **Data Preprocessing**:
   - Handle missing values based on patterns observed
   - Encode categorical variables appropriately
   - Consider scaling/normalizing numerical features
   - Implement feature engineering ideas (car age, equipment count, etc.)

2. **Feature Selection**:
   - Use correlation analysis and feature importance to select relevant predictors
   - Consider dimensionality reduction for high-cardinality features

3. **Model Selection**:
   - Try regression models (Linear Regression, Ridge, Lasso)
   - Ensemble methods (Random Forest, Gradient Boosting)
   - Consider advanced models like XGBoost, LightGBM

4. **Evaluation Strategy**:
   - Cross-validation approach
   - Metrics to use (RMSE, MAE, R²)
   - Baseline model to compare against